In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import scipy.stats as stats
import scipy.optimize as spo
from scipy.optimize import minimize
from helpers_similarity import *
from helpers_optimization import *

In [2]:
# standard deviation of spherical Gaussian distribution
mu = 0.0 
sigma_u = 1.0
sigma_v = 1.0
sigma = 0.2#[0, 0.2, 0.4, 0.6, 0.8, 1] 

D = 2 #dimension
N = 10 #nbr of users = 10
M = 10 #nbr of movies : M=5*alpha M = 10, 30, 100, 300, 1000
alpha = 1 
alpha_vals = [1, 3, 10] #[1,5,10,50,100]
k_best = 10 # 
nb_iter = 100 #1000

optimize the objective function by minimizing it w.r.t U and V respectively 

In [7]:
def optimization_fn(alpha):
    M_cols = M*alpha
    U, V, R = generate_U_V_R(mu, sigma_u, sigma_v, sigma, alpha, D, N, M)

    U0 = np.random.normal(mu, sigma_u, D*N) 
    V0 = np.random.normal(mu, sigma_v, D*M*alpha) #np.random.randn(D*M_cols) random guess for V0
    U_res = minimize(objective_function_U, U0, args=(R, V, sigma_u, sigma_v, sigma, D, N))
    V_res = minimize(objective_function_V, V0, args=(R, U, sigma_u, sigma_v, sigma, alpha, D, N))

    U_result = 0
    if U_res.success :
        U_result = U_res.x.reshape((D, N))
    else :
        print('No minimum found for U')
        U_result = np.zeros((D, N))
    diff_norm_U = frob(U, U_result, N)
    
    V_result = 0
    if V_res.success :
        V_result = V_res.x.reshape((D, M_cols))
    else :
        print('No minimum found for V')
        V_result = np.zeros((D, M_cols))
    diff_norm_V = frob(V, V_result, M_cols)
    #diff_norm = procrustes_norm(V, V_result, M)
    return diff_norm_U, diff_norm_V


compute average Difference (in Frobenius/ Procrustes norm) of U, U_result and V, V_result for different values of M

In [8]:
for i in range(len(alpha_vals)):
    avg_U = 0
    avg_V = 0
    for j in range(nb_iter):
        diff_U, diff_V = optimization_fn(alpha_vals[i])
        avg_U += diff_U
        avg_V += diff_V
    avg_U = avg_U/nb_iter
    avg_V = avg_V/nb_iter
    print(f'Average Difference of Frobenius norm of U and U_result for N= {N}: {avg_U}')
    print(f'Average Difference of Frobenius norm of V and V_result for M= {M*alpha_vals[i]}: {avg_V}')



Average Difference of Frobenius norm of U and U_result for N= 10: 0.1076184991898389
Average Difference of Frobenius norm of V and V_result for M= 10: 0.10861168999261972
Average Difference of Frobenius norm of U and U_result for N= 10: 0.055380219644697325
Average Difference of Frobenius norm of V and V_result for M= 30: 0.10782459147263383
Average Difference of Frobenius norm of U and U_result for N= 10: 0.028545516022403527
Average Difference of Frobenius norm of V and V_result for M= 100: 0.10926565118837772
